In [17]:
import pandas as pd
import nltk

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

import simplejson as json

import numpy as np

# Introdução
<div align="justify">
   <p> 	&nbsp;	&nbsp;	&nbsp;Esse documento tem como objetivo analisar classificação de notícias vs sátiras. O intuito por trás dessa analise é tentar verificar as diferenças entre estes dois tipos de informação. O objetivo por trás disso é tentar a partir de uma base claramente falsa(sátiras), verificar nuances em relação a uma base de notícia reais. Isto pode ser interessante, pois como sátiras e fake news são notícias falsas, pode ser possível que ao se treinar um modelo utilizando sátiras(que possui base de dados maior disponível), possa ser possível identificar fake news. Contudo, não analisaremos esta possivel correlação neste documento.</p>
    <br>
   	&nbsp;	&nbsp;	&nbsp;A base de dados que iremos utilizar está disponível em: <a href="https://drive.google.com/file/d/0B7MeyftEk73Wbk1MQTJzeWVtRU1FaDBTb3dwTy1yVkc1MHVz/view?usp=sharing"> base de dados</a>. Nela encontraremos 8748 notícias, sendo elas 4374 categorizadas do tipo 'REAL' e 4374 com a categorização 'FAKE'. As notícias reais foram retiradas de sites de notícias com fontes confiáveis, enquanto as notificas do tipo fake foram retiradas de sites de humor, principalmente do <a href="http://sensacionalista.com.br"> sensacionalista </a>. Para a limpeza de dados, foi decidido utilizar funções para remover tudo que não era alfabético ou que fosse uma <a href="https://pt.wikipedia.org/wiki/Palavra_vazia" > palavra vazia. </a>
</div>


# Funções auxiliares:

Para realizar esta análise foram criadas algumas funções auxiliares que estão descritas a seguir:

In [18]:
#Função para remover as pontuações, caracteres não alfabeticos dos documentos e stopwords
def removeNotRelevantWordsFromRow(row):
    stopword = stopwords.words('portuguese')
    stopword.append("é")
    words = nltk.word_tokenize(row)
    words= [word.lower() for word in words if word.isalpha()]
    wordsFormatted = [word.lower() for word in words if word not in stopword]

    return ' '.join(wordsFormatted)


In [19]:
def removeNotRelevantWordsFromDF(df):
    auxDF = df
    size = len(auxDF)
    for index in range(size):
        auxDF.text[index] = removeNotRelevantWordsFromRow(auxDF.text[index])
    
    return auxDF

In [20]:
def getDataFrameFromCSV():
    df = pd.read_csv('base_dados.csv')
    return df

In [21]:
def find_ngrams(input_list, n):
    lista = list(zip(*[input_list[i:] for i in range(n)]))
    
    if (len(lista) == 0):
        return ""
    
    return lista

In [22]:
def createLabelIDColuna(row):
    if (row == 'FAKE'):
        return 0
    
    return 1

In [23]:
def divideFakeAndReal(df):
    return df[df.label == 'REAL'], df[df.label == 'FAKE']

In [24]:
def getTfIDFAndFeaturesFromDf(df):
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=10, norm='l2', encoding='utf-8', ngram_range=(1, 2))
    features = tfidf.fit_transform(df.text).toarray()
    return tfidf, features

In [25]:
def getWordsRankingFromTFIDF(tfidf, features):
    terms = tfidf.get_feature_names()

    # sum tfidf frequency of each term through documents
    sums = features.sum(axis=0)
    data = []

    for col, term in enumerate(terms):
        data.append( (term, sums[col] ))

    ranking = pd.DataFrame(data, columns=['term','rank'])
    ranking.sort_values('rank',inplace=True, ascending=False)
    return ranking

# Analisando as palavras e bigramas mais relevantes nos dataframes:
    Nesta parte do projeto, iremos analisar as palavras e os bigramas mais importantes do conjunto de dados REAL e do conjunto de dados FAKE. Essa relevância será baseada e ranqueada com base no TFIDF.
    

In [26]:
df = getDataFrameFromCSV()
realDF, fakeDF = divideFakeAndReal(df)
#Fix Index
realDF.index = range(4375)

In [27]:
realDF = removeNotRelevantWordsFromDF(realDF)
fakeDF = removeNotRelevantWordsFromDF(fakeDF)

C:\Users\Rafael Albuquerque\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Rafael Albuquerque\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
tfidfREAL, featuresREAL = getTfIDFAndFeaturesFromDf(realDF)
tfidfFAKE, featuresFAKE = getTfIDFAndFeaturesFromDf(fakeDF)

A partir da função shape nas features, obtemos a seguinte tabela:

<table> 
    <thead> 
        <th> # </th> 
        <th> Textos </th> 
        <th> Features</th> 
    </thead> 
    <tbody> 
        <tr>
            <td> REAL </td> 
            <td> 4374 </td> 
            <td> 21726 </td> 
        </tr>
        <tr>
          <td> FAKE </td> 
          <td> 4374 </td> 
          <td> 5432</td> 
        </tr>
    </tbody>
</table>

Percebemos então que nas 4374 linhas de texto de ambos os dataframes, enquanto o dataframe REAL tem 21726 features(conjuto de palavras e bigramas), o fake possui apenas 5432 para as mesmas 4374 linhas. 

In [29]:
rankingFAKE = getWordsRankingFromTFIDF(tfidfFAKE, featuresFAKE)
rankingREAL = getWordsRankingFromTFIDF(tfidfREAL, featuresREAL)

Agora aqui temos as 10 palavras e os bigramas mais relevantes(com base no TFIDF) do conjunto de dados fake:

In [30]:
rankingFAKE.head(10)

,term,rank
1591,disse,112.573469
4680,ser,82.607144
4015,presidente,73.242460
1549,dilma,72.293398
2477,hoje,68.034295
5159,vai,64.176920
4612,segundo,63.594587
639,brasil,62.723910
4484,rio,62.612771
4908,ter,61.731067


In [31]:
rankingFAKE['term'] = rankingFAKE['term'].map(lambda text: find_ngrams(text.split(" "), 2))
rankingFAKE[rankingFAKE.term != ""].head(10)

,term,rank
4486,"[(rio, janeiro)]",43.161893
1553,"[(dilma, rousseff)]",31.267293
3136,"[(michel, temer)]",29.115406
4020,"[(presidente, dilma)]",26.926771
1745,"[(eduardo, cunha)]",23.926910
4358,"[(redes, sociais)]",22.303762
3881,"[(polícia, federal)]",19.971674
508,"[(aécio, neves)]",19.358600
4976,"[(todo, mundo)]",19.298901
3851,"[(pode, ser)]",18.573038


Agora aqui temos as 10 palavras mais relevantes(com base no TFIDF) do conjunto de dados real:
ranking


In [32]:
rankingREAL.head(10)

,term,rank
15794,presidente,92.121205
6493,disse,83.909771
19847,temer,82.148183
8443,federal,76.274089
18865,ser,73.927833
9410,governo,73.842090
19235,sobre,71.387218
545,afirmou,69.924720
18557,segundo,68.317896
12557,ministro,65.393654


In [33]:
rankingREAL['term'] = rankingREAL['term'].map(lambda text: find_ngrams(text.split(" "), 2))
rankingREAL[rankingREAL.term != ""].head(10)

,term,rank
11252,"[(lava, jato)]",64.591437
12435,"[(michel, temer)]",48.486825
19665,"[(supremo, tribunal)]",40.987895
12652,"[(ministério, público)]",40.897214
20558,"[(tribunal, federal)]",40.839011
6269,"[(dilma, rousseff)]",38.248806
13785,"[(operação, lava)]",34.691884
15840,"[(presidente, michel)]",33.339170
7076,"[(eduardo, cunha)]",25.615235
8520,"[(federal, stf)]",25.553947


Podemos notar que apesar de existir algumas features com iguais relevâncias nos dois conjuntos de dados, em geral as features de maior relevância diferem de um conjunto para o outro.

# Analisando a classificação de notícias vs sátiras:


In [34]:
df = getDataFrameFromCSV()
df['label_id'] = df['label'].map(lambda label: createLabelIDColuna(label))

tfidf, features = getTfIDFAndFeaturesFromDf(df)
labels = df['label_id']

Nesta próxima etapa compareremos quatro métodos de classificação:
 - Regressão
 - Naive Bayes
 - Máquina de vetores de suporte (SVM)
 - Random Forest

Para comparar, utilizaremos <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html"> validação cruzada </a> disponivel pelo sklearn

In [35]:

models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [36]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 0.984909
LogisticRegression        0.977823
MultinomialNB             0.955881
RandomForestClassifier    0.936563
Name: accuracy, dtype: float64

Podemos ver, a partir dos dados acima, que a acurácia dos métodos se mostrou bastante parecida, com apenas uma leve vantagem para o SVM(Descrito como LinearSVC). Mostrando assim que há indícios que aspectos textuais de sátiras são bem diferentes de aspectos textuais de notícias verdadeiras.

# Pequena aplicação:

Para esta analise, foi criada uma pequena aplicação, onde o leitor pode colocar um texto e escolher o classificador. A partir disso será retornado uma predição que informará se a notícia é falsa ou verdadeira.

In [64]:
def trainModel(df):
    X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label_id'], random_state = 0)
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    return X_train_tfidf, y_train, count_vect

In [65]:
def getClassifier(option, X_train_tfidf, y_train):
    if (option == "regress"):
        clf = LogisticRegression(random_state=0).fit(X_train_tfidf, y_train)
    elif (option == "naive"):
        clf = MultinomialNB().fit(X_train_tfidf, y_train)
    elif (option == "svm"):
        clf = LinearSVC().fit(X_train_tfidf, y_train)
    elif (option == "rf"):
        clf = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0).fit(X_train_tfidf, y_train)
    return clf

In [66]:
def checkData(data, method):
    df = getDataFrameFromCSV()
    df['label_id'] = df['label'].map(lambda label: createLabelIDColuna(label))
    X_train_tfidf, y_train, count_vect = trainModel(df)
    clf = getClassifier(method, X_train_tfidf, y_train)
    dataArray = []
    dataArray.append(data)
    result = clf.predict(count_vect.transform(dataArray))
    
    if (len(result) > 0):
        if (result[0] == 0):
            print ('FAKE')
        elif (result[0] == 1):
            print ('REAL')


In [67]:
from IPython.core.display import HTML
HTML("""
<html>
<head>
    <script>
    function handle_output(out){
      alert(out.content.text);

    }

    
    function predict() {
      var element = document.getElementById("options");
      var textArea = document.getElementById("textArea");
      var content = textArea.value;
      var option = element.options[element.selectedIndex].value;
      var callbacks = {iopub : { output : handle_output}}
      var kernel = IPython.notebook.kernel;
      var command = "checkData('"+content+ "','" + option + "')"
      var msg_id = kernel.execute(command, callbacks);

    }
    </script>
</head>
<body>
  <select id="options">
    <option value="regress">Regressão</option>
    <option value="naive">Naive Bayes</option>
    <option value="svm">SVM</option>
    <option value="rf">Random Forest</option>
  </select>
  <button onclick="predict()">Predict</button>
  <div>
    <textarea id="textArea" rows="20" cols="80">
    </textarea>
  </div>
</body>
</html>
""")

<a href="https://drive.google.com/file/d/1qjzdlTX4jJp60jm3EnOBqmNflZFYhhSR/view?usp=sharing"> <b>Video demonstrativo </b></a>

# Conclusão e trabalhos futuros

   Conclui-se, então, que em todos os classsificadores utilizados, há sim diferença entre uma notícia do tipo sátira e uma do tipo real. Também podemos notar que há indicios que as palavras utilizadas neste tipo de notícia(sátira) são bem diferentes das utilizadas em notícias reais. 
   <br>
   <br>
    Para futuros estudos, pensa-se se em aumentar a base de dados e também incluir além do texto da notícia, o título. Também se pensa em analisar/classificar individualmente as palavras e os bigramas. Por último, pode vir a ser interessante analisar a correlação entre fake news e sátiras, pois, caso haja uma alta correlação, o modelo proposto pode vir a ser uma boa alternativa para se descobrir fake news.